In [2]:
from osgeo import gdal, osr
import os

In [3]:
# Define the input and output folder paths
input_folder = "/Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade"  # Change to your folder path
output_folder = '/Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected'  # Replace with your output folder path

In [30]:
import os
from osgeo import gdal, osr



# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to project a single file
def project_file(input_file, output_file):
    # Open the input TIFF file
    dataset = gdal.Open(input_file)

    if dataset is None:
        print(f"Error opening {input_file}")
        return

    # Set up geotransformation parameters based on filename (modify as per your logic)
    origin_x = float(input_file.split('/')[-1][5:9] + '00')  # Replace with the correct value from the image
    origin_y = float(input_file.split('/')[-1][9:14] + '00') + 1000  # Replace with the correct value from the image

    # Pixel size
    pixel_width = 0.5
    pixel_height = -0.5  # Negative for georeferenced rasters (top-to-bottom)

    # Define the geotransform (this defines where the image fits in the real world)
    geotransform = (origin_x, pixel_width, 0, origin_y, 0, pixel_height)

    # Create a new GeoTIFF with georeferencing
    driver = gdal.GetDriverByName('GTiff')
    out_dataset = driver.CreateCopy(output_file, dataset, 0)

    # Apply the geotransformation to the dataset
    out_dataset.SetGeoTransform(geotransform)

    # Set the coordinate system (assumed to be EPSG:26917 for UTM Zone 17N, adjust if necessary)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(26917)  # Replace with the appropriate EPSG code if needed
    out_dataset.SetProjection(srs.ExportToWkt())

    # Flush data to disk and close the dataset
    out_dataset.FlushCache()
    out_dataset = None
    dataset = None

    print(f"GeoTIFF saved to {output_file}")

# Iterate over files in the input folder (no subfolders)
for filename in os.listdir(input_folder):
    if filename.endswith(".tif"):  # Process only .tif files
        input_file = os.path.join(input_folder, filename)
        output_file = os.path.join(output_folder, filename.replace('.tif', '_shade.tif'))
        project_file(input_file, output_file)

# After processing all files, merge the output GeoTIFFs into one file
# Collect all output files
output_files = [os.path.join(output_folder, filename.replace('.tif', '_shade.tif'))
                for filename in os.listdir(input_folder)
                if filename.endswith('.tif')]


GeoTIFF saved to Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km175050508502020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km175090509502020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km175020509402020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km175100510102020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km174970509302020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km174970508802020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km174970508602020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km17513050940202

In [31]:
# Define the name of the VRT and GeoTIFF output files
vrt_output_file = os.path.join(output_folder, 'merged_output.vrt')
tif_output_file = os.path.join(output_folder, 'merged_output.tif')

# Step 1: Create a virtual mosaic using gdalbuildvrt
print("Creating a virtual mosaic using gdalbuildvrt...")
gdal.BuildVRT(vrt_output_file, output_files)

# Step 2: Convert the virtual mosaic to a GeoTIFF using gdal_translate
print("Converting the virtual mosaic to GeoTIFF using gdal_translate...")
gdal.Translate(tif_output_file, vrt_output_file, format='GTiff')

print("Mosaic creation complete! Output saved at:", tif_output_file)

Creating a virtual mosaic using gdalbuildvrt...
Converting the virtual mosaic to GeoTIFF using gdal_translate...
Mosaic creation complete! Output saved at: Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/merged_output.tif


In [7]:
# Define the input merged file and the output file path
merged_output_file = '/Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/merged_output.tif'  # Replace with the path to your merged GeoTIFF file
downsampled_output_file = '/Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/merged_output_downsampled_2.tif'  # Output file path for the downsampled version

# Open the merged GeoTIFF file
dataset = gdal.Open(merged_output_file)

# Get the original resolution (pixel size)
original_width = dataset.RasterXSize
original_height = dataset.RasterYSize

# Calculate new resolution (half of the original resolution)
new_width = original_width // 4
new_height = original_height // 4

# Create options for gdal.Warp (downsampling and compression)
warp_options = gdal.WarpOptions(
    width=new_width,  # Downsample to half the original width
    height=new_height,  # Downsample to half the origina`l height
    format='GTiff',  # Output format
    creationOptions=['COMPRESS=LZW', 'TILED=YES']  # Apply LZW compression and tiling
)

# Downsample and compress the merged output file
print(f"Downsampling and compressing {merged_output_file}...")
gdal.Warp(downsampled_output_file, dataset, options=warp_options)

print(f"Downsampled and compressed file saved to: {downsampled_output_file}")


Downsampling and compressing /Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/merged_output.tif...
Downsampled and compressed file saved to: /Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/merged_output_downsampled_2.tif


# Project and Merge to 5x5

In [5]:
import os
from osgeo import gdal, osr

# Define the input and output folder paths
input_folder = '/Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade'  # Replace with your input folder path
output_folder = '/Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/'  # Replace with your output folder path
merged_folder = 'W/hillshade/Merged/'  # Folder to store the merged 5x5 blocks

# Create the output and merged folder if they don't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

if not os.path.exists(merged_folder):
    os.makedirs(merged_folder)

# Function to project a single file
def project_file(input_file, output_file):
    dataset = gdal.Open(input_file)
    if dataset is None:
        print(f"Error opening {input_file}")
        return

    origin_x = float(input_file.split('/')[-1][5:9] + '00')
    origin_y = float(input_file.split('/')[-1][9:14] + '00') + 1000

    pixel_width = 0.5
    pixel_height = -0.5

    geotransform = (origin_x, pixel_width, 0, origin_y, 0, pixel_height)

    driver = gdal.GetDriverByName('GTiff')
    out_dataset = driver.CreateCopy(output_file, dataset, 0)

    out_dataset.SetGeoTransform(geotransform)

    srs = osr.SpatialReference()
    srs.ImportFromEPSG(26917)
    out_dataset.SetProjection(srs.ExportToWkt())

    out_dataset.FlushCache()
    out_dataset = None
    dataset = None

    print(f"GeoTIFF saved to {output_file}")

# Project and process each file
for filename in sorted(os.listdir(input_folder)):
    if filename.endswith(".tif"):
        input_file = os.path.join(input_folder, filename)
        output_file = os.path.join(output_folder, filename.replace('.tif', '_shade.tif'))
        project_file(input_file, output_file)


GeoTIFF saved to /Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km174920508402020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to /Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km174920508502020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to /Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km174920508602020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to /Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km174920508702020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to /Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km174920508802020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to /Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km174920508902020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to /Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km174920509002020LLAKENIPISSING_shade_shade.tif
GeoTIFF saved to /Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/1km174930

KeyboardInterrupt: 

In [ ]:
# Group the output GeoTIFFs into 5x5 blocks and merge
output_files = [os.path.join(output_folder, filename.replace('.tif', '_shade.tif'))
                for filename in os.listdir(input_folder)
                if filename.endswith('.tif')]

# Sort files based on names to maintain grid order (adjust sorting logic as needed)
output_files.sort()

block_size = 5
block_count = 0
for i in range(0, len(output_files), block_size**2):  # Step through in 5x5 blocks
    block_files = output_files[i:i + block_size**2]
    if len(block_files) == 0:
        break
    
    # Define the name of the merged output file for each block
    block_output_file = os.path.join(merged_folder, f'merged_block_{block_count}.tif')
    
    # Use gdal.Warp to merge the files in the block
    print(f"Merging files into block {block_count} GeoTIFF...")
    gdal.Warp(block_output_file, block_files, format='GTiff')
    
    print(f"Merged block {block_count} GeoTIFF saved to {block_output_file}")
    
    block_count += 1

# KMZ

In [38]:
# def convert_raster_to_kmz(input_raster, output_kmz):
#     # Register all available GDAL drivers
#     gdal.AllRegister()

#     # Open the input raster file
#     raster = gdal.Open(input_raster)
#     if raster is None:
#         print(f"Failed to open file: {input_raster}")
#         return

#     # Ensure the raster has a spatial reference and is georeferenced
#     if raster.GetProjection() == '':
#         print("The input raster is not georeferenced.")
#         return
    
#     # Define the driver for KMZ output
#     driver = gdal.GetDriverByName('KMLSUPEROVERLAY')
#     if driver is None:
#         print("KMLSUPEROVERLAY driver not found")
#         return

#     try:
#         # Create and write to the output KMZ file
#         driver.CreateCopy(output_kmz, raster)
#         print(f"KMZ file converted successfully: {output_kmz}")
#     except Exception as e:
#         print(f"Error occurred: {e}")
#     finally:
#         # Close the dataset
#         raster = None

In [39]:
# # Convert the merged output to KMZ
# output_kmz = os.path.join(output_folder, 'merged_output.kmz')
# print("Converting merged GeoTIFF to KMZ...")
# convert_raster_to_kmz(merged_output_file, output_kmz)

Converting merged GeoTIFF to KMZ...
KMZ file converted successfully: Elora/hillshade/Projected/merged_output.kmz


In [2]:
# # Function to convert individual GeoTIFF raster files to KML super overlay and add them to a KMZ
# def add_tile_to_kmz(input_raster, output_kmz_folder):
#     # Register all available GDAL drivers
#     gdal.AllRegister()

#     # Open the input raster file
#     raster = gdal.Open(input_raster)
#     if raster is None:
#         print(f"Failed to open file: {input_raster}")
#         return

#     # Ensure the raster has a spatial reference and is georeferenced
#     if raster.GetProjection() == '':
#         print("The input raster is not georeferenced.")
#         return

#     # Define the driver for KML super overlay output
#     driver = gdal.GetDriverByName('KMLSUPEROVERLAY')
#     if driver is None:
#         print("KMLSUPEROVERLAY driver not found")
#         return

#     try:
#         # Create a folder to store intermediate KML and raster files if it doesn't exist
#         if not os.path.exists(output_kmz_folder):
#             os.makedirs(output_kmz_folder)

#         # Define the output path for the KML super overlay
#         tile_name = os.path.basename(input_raster).replace('.tif', '')
#         output_tile_kml = os.path.join(output_kmz_folder, f"{tile_name}.kmz")

#         # Create and write the KML super overlay to the KMZ file
#         driver.CreateCopy(output_tile_kml, raster)
#         print(f"Tile {tile_name} converted successfully to KMZ: {output_tile_kml}")
        
#     except Exception as e:
#         print(f"Error occurred: {e}")
#     finally:
#         # Close the dataset
#         raster = None



In [7]:
import os
from osgeo import gdal
import zipfile
import xml.etree.ElementTree as ET

# Function to convert individual GeoTIFF raster files to KML super overlay
def convert_raster_to_kml(input_raster, output_kml_folder):
    # Register all available GDAL drivers
    gdal.AllRegister()

    # Open the input raster file
    raster = gdal.Open(input_raster)
    if raster is None:
        print(f"Failed to open file: {input_raster}")
        return

    # Ensure the raster has a spatial reference and is georeferenced
    if raster.GetProjection() == '':
        print(f"The input raster {input_raster} is not georeferenced.")
        return

    # Define the driver for KML super overlay output
    driver = gdal.GetDriverByName('KMLSUPEROVERLAY')
    if driver is None:
        print("KMLSUPEROVERLAY driver not found")
        return

    try:
        # Create the folder for the KML output
        tile_name = os.path.basename(input_raster).replace('.tif', '')
        tile_output_folder = os.path.join(output_kml_folder, tile_name)

        if not os.path.exists(tile_output_folder):
            os.makedirs(tile_output_folder)

        # Define the output KML path (it will generate both KML and image tiles in the folder)
        output_tile_kml = os.path.join(tile_output_folder, f"{tile_name}.kml")

        # Write the KML super overlay to a folder (this also creates associated images)
        driver.CreateCopy(output_tile_kml, raster)
        print(f"Tile {tile_name} converted successfully to KML and images: {output_tile_kml}")

    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        # Close the dataset
        raster = None

# Function to create a master KML file that references all individual KML files
def create_master_kml(kml_folder, master_kml_path):
    kml_namespace = "http://www.opengis.net/kml/2.2"
    ET.register_namespace("", kml_namespace)

    # Create the root KML element
    kml = ET.Element("{%s}kml" % kml_namespace)
    document = ET.SubElement(kml, "Document")

    # Loop through each subfolder (each tile)
    for tile_name in os.listdir(kml_folder):
        tile_folder = os.path.join(kml_folder, tile_name)
        if os.path.isdir(tile_folder):
            tile_kml_file = os.path.join(tile_folder, f"{tile_name}.kml")
            if os.path.exists(tile_kml_file):
                # Create a NetworkLink to the tile KML file
                network_link = ET.SubElement(document, "NetworkLink")
                name = ET.SubElement(network_link, "name")
                name.text = tile_name
                link = ET.SubElement(network_link, "Link")
                href = ET.SubElement(link, "href")
                href.text = f"{tile_name}/{tile_name}.kml"  # Relative path inside the KMZ
                view_refresh_mode = ET.SubElement(link, "viewRefreshMode")
                view_refresh_mode.text = "onRegion"
    
    # Write the master KML to a file
    tree = ET.ElementTree(kml)
    tree.write(master_kml_path, encoding="UTF-8", xml_declaration=True)
    print(f"Master KML file created: {master_kml_path}")

# Function to zip all KML files and associated images into a single KMZ
def create_kmz(kml_folder, kmz_file, master_kml_path):
    with zipfile.ZipFile(kmz_file, 'w', zipfile.ZIP_DEFLATED) as kmz:
        # Add the master KML file as 'doc.kml' at the root
        kmz.write(master_kml_path, 'doc.kml')

        # Loop through each subfolder inside the KML folder
        for root, _, files in os.walk(kml_folder):
            for file in files:
                file_path = os.path.join(root, file)
                # Exclude the master KML file (already added as 'doc.kml')
                if file_path == master_kml_path:
                    continue
                # Calculate the relative path inside the KMZ
                relative_path = os.path.relpath(file_path, kml_folder)
                kmz.write(file_path, relative_path)
    print(f"Master KMZ file created successfully: {kmz_file}")

# Define the input and output folder paths
input_folder = '/Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/'  # Folder with the individual projected GeoTIFF tiles
output_kml_folder = '/Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/KML_tiles/'  # Folder to store individual KML files and image tiles

# Ensure the output folder for KML files exists
if not os.path.exists(output_kml_folder):
    os.makedirs(output_kml_folder)

# Iterate over the individual GeoTIFF tiles and convert each to KML
for filename in os.listdir(input_folder):
    if filename.endswith("_shade.tif"):  # Process only the projected .tif files
        input_raster = os.path.join(input_folder, filename)
        print(f"Converting {filename} to KML...")
        convert_raster_to_kml(input_raster, output_kml_folder)

# Define the path for the master KML file
master_kml_path = os.path.join(output_kml_folder, 'master.kml')

# Create the master KML file that references all individual KML files
create_master_kml(output_kml_folder, master_kml_path)

# Define the final KMZ output file path
kmz_file = os.path.join(output_kml_folder, 'merged_output.kmz')

# Create the final KMZ by zipping the master KML and supporting files
create_kmz(output_kml_folder, kmz_file, master_kml_path)



Converting 1km174950508902020LLAKENIPISSING_shade_shade.tif to KML...
Tile 1km174950508902020LLAKENIPISSING_shade_shade converted successfully to KML and images: /Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/KML_tiles/1km174950508902020LLAKENIPISSING_shade_shade/1km174950508902020LLAKENIPISSING_shade_shade.kml
Converting 1km175000509502020LLAKENIPISSING_shade_shade.tif to KML...
Tile 1km175000509502020LLAKENIPISSING_shade_shade converted successfully to KML and images: /Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/KML_tiles/1km175000509502020LLAKENIPISSING_shade_shade/1km175000509502020LLAKENIPISSING_shade_shade.kml
Converting 1km175020509302020LLAKENIPISSING_shade_shade.tif to KML...
Tile 1km175020509302020LLAKENIPISSING_shade_shade converted successfully to KML and images: /Users/shuyang/Data/DTM/LakeNipissing-DTM-A/hillshade/Projected/KML_tiles/1km175020509302020LLAKENIPISSING_shade_shade/1km175020509302020LLAKENIPISSING_shade_shade.kml
Converting

In [ ]:


# # Define the input and output folder paths
# input_folder = 'Elora/hillshade'  # Replace with your input folder path
# output_folder = 'Elora/hillshade/Projected/'  # Replace with your output folder path

# # Create the output folder if it doesn't exist
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# # Function to project a single file
# def project_file(input_file, output_file):
#     # Open the input TIFF file
#     dataset = gdal.Open(input_file)

#     if dataset is None:
#         print(f"Error opening {input_file}")
#         return

#     # Set up geotransformation parameters based on filename (modify as per your logic)
#     origin_x = float(input_file.split('/')[-1][5:9] + '00')  # Replace with the correct value from the image
#     origin_y = float(input_file.split('/')[-1][9:14] + '00') + 1000  # Replace with the correct value from the image

#     # Pixel size
#     pixel_width = 0.5
#     pixel_height = -0.5  # Negative for georeferenced rasters (top-to-bottom)

#     # Define the geotransform (this defines where the image fits in the real world)
#     geotransform = (origin_x, pixel_width, 0, origin_y, 0, pixel_height)

#     # Create a new GeoTIFF with georeferencing
#     driver = gdal.GetDriverByName('GTiff')
#     out_dataset = driver.CreateCopy(output_file, dataset, 0)

#     # Apply the geotransformation to the dataset
#     out_dataset.SetGeoTransform(geotransform)

#     # Set the coordinate system (assumed to be EPSG:26917 for UTM Zone 17N, adjust if necessary)
#     srs = osr.SpatialReference()
#     srs.ImportFromEPSG(26917)  # Replace with the appropriate EPSG code if needed
#     out_dataset.SetProjection(srs.ExportToWkt())

#     # Flush data to disk and close the dataset
#     out_dataset.FlushCache()
#     out_dataset = None
#     dataset = None

#     print(f"GeoTIFF saved to {output_file}")

# # Iterate over files in the input folder (no subfolders)
# for filename in os.listdir(input_folder):
#     if filename.endswith(".tif"):  # Process only .tif files
#         input_file = os.path.join(input_folder, filename)
#         output_file = os.path.join(output_folder, filename.replace('.tif', '_shade.tif'))
#         project_file(input_file, output_file)


GeoTIFF saved to Elora/hillshade/Projected/1km175460483702018LLAKEERIE_rescaled_shade.tif
GeoTIFF saved to Elora/hillshade/Projected/1km175450483602018LLAKEERIE_rescaled_shade.tif
GeoTIFF saved to Elora/hillshade/Projected/1km175440483802018LLAKEERIE_rescaled_shade.tif
GeoTIFF saved to Elora/hillshade/Projected/1km175440483402018LLAKEERIE_rescaled_shade.tif
GeoTIFF saved to Elora/hillshade/Projected/1km175470483502018LLAKEERIE_rescaled_shade.tif
GeoTIFF saved to Elora/hillshade/Projected/1km175430483702018LLAKEERIE_rescaled_shade.tif
GeoTIFF saved to Elora/hillshade/Projected/1km175450483702018LLAKEERIE_rescaled_shade.tif
GeoTIFF saved to Elora/hillshade/Projected/1km175460483602018LLAKEERIE_rescaled_shade.tif
GeoTIFF saved to Elora/hillshade/Projected/1km175470483402018LLAKEERIE_rescaled_shade.tif
GeoTIFF saved to Elora/hillshade/Projected/1km175470483802018LLAKEERIE_rescaled_shade.tif
GeoTIFF saved to Elora/hillshade/Projected/1km175430483602018LLAKEERIE_rescaled_shade.tif
GeoTIFF sa